In [ ]:
import pandas as pd
import numpy as np

import constants as cons

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.graph_objs as go
import plotly.offline as py
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
#import lightgbm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
sns.set(style='white', context='notebook', palette='deep') 

import matplotlib.style as style
style.use('fivethirtyeight')
import time

In [ ]:
df = pd.read_csv("data/nyc-rolling-sales.csv", index_col=0)
df.head()

BOROUGH: 
A digit code for the borough the property is located in.

- Manhattan = 1

- Bronx = 2

- Brooklyn = 3

- Queens = 4

- Staten Island = 5

BLOCK; LOT: 

- The combination of borough, block, and lot forms a unique key for property in New York City.


BUILDING CLASS AT PRESENT and BUILDING CLASS AT TIME OF SALE: 

- The type of building at various points in time.


** Note : 0 dollar sales are actually transfers of ownership between parties.

In [ ]:

df_raw = df.copy()

In [ ]:
###################################### EXPLORE ######################################

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
###################################### FEATURE ENGINEERING ######################################

In [ ]:
df['BUILDING AGE'] = 2017 - df['YEAR BUILT']
df['BOROUGH'] = df['BOROUGH'].map(cons.borough_encoding)
df['sale_year'] = pd.DatetimeIndex(df[cons.date_column]).year
df['sale_month'] = pd.DatetimeIndex(df[cons.date_column]).month

In [ ]:
#### DATA TYPE CONVERSIONS

df[cons.date_column] = pd.to_datetime(df[cons.date_column], errors='coerce') 

for col in cons.numeric_columns: 
    df[col] = pd.to_numeric(df[col], errors='coerce')

for col in cons.categorical_columns: 
    df[col] = df[col].astype(object)

In [ ]:
#### MISSING DATA ANALYSIS

df.replace(' ',np.nan, inplace=True)
round(df.isna().sum() /len(df) *100,2)

In [ ]:
#################### FILLING MISSING VALUES #########################

In [ ]:
print(df[(df['LAND SQUARE FEET'].isnull()) & (df['GROSS SQUARE FEET'].notnull())].shape[0])

print(df[(df['LAND SQUARE FEET'].notnull()) & (df['GROSS SQUARE FEET'].isnull())].shape[0])

In [ ]:
print(df[(df['LAND SQUARE FEET'] == 0) & (df['GROSS SQUARE FEET'] != 0)].shape[0])
print(df[(df['LAND SQUARE FEET'] != 0) & (df['GROSS SQUARE FEET'] == 0)].shape[0])

In [ ]:
imputation = df[['LAND SQUARE FEET','GROSS SQUARE FEET']]
imputation[(imputation['LAND SQUARE FEET'].isnull()) & (imputation['GROSS SQUARE FEET'].notnull())]

imputation.dropna(inplace = True)

In [ ]:
# Regression model for predicting Gross Square Feet

X = imputation['LAND SQUARE FEET']
y = imputation['GROSS SQUARE FEET']

lm = LinearRegression()
model = lm.fit(X.values.reshape(-1,1),y.values.reshape(-1,1))

print("predicting Gross Square Feet")
print("Intercept: ", model.intercept_)
print("Coefficients: ", model.coef_)
print("Regression Model: Y={}+{}X".format(model.intercept_,model.coef_))
print("Rsquare: ", model.score(X.values.reshape(-1,1),y.values.reshape(-1,1)))


# Regression model for predicting Land Square Feet

X = imputation['GROSS SQUARE FEET']
y = imputation['LAND SQUARE FEET']

lm = LinearRegression()
model = lm.fit(X.values.reshape(-1,1),y.values.reshape(-1,1))

print("predicting Land Square Feet")
print("Intercept: ", model.intercept_)
print("Coefficients: ", model.coef_)
print("Regression Model: Y={}+{}X".format(model.intercept_,model.coef_))
print("Rsquare: ", model.score(X.values.reshape(-1,1),y.values.reshape(-1,1)))

In [ ]:
# Gross square feet estimator

def imputer_gross(x):
    return x * 0.64779857 + 1822.76092513

# land square feet estimator

def imputer_land(x):
    return x * 0.6375468 + 1107.3023519

# Imputing missing values of theese columns.

df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].mask((df['LAND SQUARE FEET'].isnull()) & (df['GROSS SQUARE FEET'].notnull()),
                                                     imputer_land(df['GROSS SQUARE FEET']))
df['GROSS SQUARE FEET'] = df['GROSS SQUARE FEET'].mask((df['LAND SQUARE FEET'].notnull()) & (df['GROSS SQUARE FEET'].isnull()),
                                                       imputer_gross(df['LAND SQUARE FEET']))

In [ ]:
del df["BUILDING CLASS AT PRESENT"] # It contains the same information with BUILDING CLASS CATEGORY
del df["ADDRESS"] # The variable 1 not required for the model
del df["ZIP CODE"] # The variable 2 not required for the model
del df["SALE DATE"] # We don't need it as we have created the year and month variables.
df.drop(["EASE-MENT","APARTMENT NUMBER"], axis=1, inplace=True) # Completely empty columns.

df = df.dropna() 
df.drop_duplicates(keep = "last", inplace=True)

In [ ]:
# Removing data where commercial + residential doesn't equal total units
df = df[df['TOTAL UNITS'] == df['COMMERCIAL UNITS'] + df['RESIDENTIAL UNITS']]

In [ ]:
df[["TOTAL UNITS", "SALE PRICE"]].groupby(['TOTAL UNITS'], as_index=False).count().sort_values(by='SALE PRICE', ascending=False)

In [ ]:
# Removing rows with TOTAL UNITS == 0 and one outlier with 2261 units
df = df[(df['TOTAL UNITS'] > 0) & (df['TOTAL UNITS'] != 2261)]

# Year cannot be 0.
df = df[df['YEAR BUILT'] != 0]
del df['YEAR BUILT']

# Square feet should not be 0.
df = df[df["LAND SQUARE FEET"] != 0]
df = df[df["GROSS SQUARE FEET"] != 0]

df.shape

In [ ]:
################ OUTLIER ANALYSIS

df_after_missing = df.copy()
round(df.describe([0.75,0.85,0.95,0.99,0.995,0.999]),3)

In [ ]:
del df["LOT"]

In [ ]:
############## VISUALIZE

sns.set_style("whitegrid")
plt.figure(figsize=(12,5))
plotd = sns.distplot(df[(df['SALE PRICE']>100) & (df['SALE PRICE'] < 5000000)]['SALE PRICE'], kde=True, bins=100)

tick_spacing=250000
plotd.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
plotd.set_xlim([-100000, 5000000])
plt.xticks(rotation=30)
plt.axvline(df[(df['SALE PRICE']>100) & (df['SALE PRICE'] < 5000000)]['SALE PRICE'].mean(), c='red')
plt.axvline(df[(df['SALE PRICE']>100) & (df['SALE PRICE'] < 5000000)]['SALE PRICE'].median(), c='blue')
plt.text(250000,0.0000012, "median")
plt.text(850000,0.0000010, "mean")
plt.show()

In [ ]:
m = df.select_dtypes(np.number)
df[m.columns]= m.round().astype('Int64')
pd.options.display.float_format = '{:,.0f}'.format
df.rename(columns = {"SALE PRICE":"sale_price"},inplace = True)

In [ ]:
df2 = df[(df['sale_price'] > 100000) & (df['sale_price'] < 5000000)]

trace0 = go.Box(
    y=df2.sale_price[df2.BOROUGH == 'Manhattan' ],
    name = 'Manhattan',
    marker = dict(
        color = 'rgb(12, 12, 140)',
    )
)
trace1 = go.Box(
    y=df2.sale_price[df2.BOROUGH ==  'Bronx' ],
    name = 'Bronx',
    marker = dict(
        color = 'rgb(8,81,156)',
    )
)
trace2 = go.Box(
    y=df2.sale_price[df2.BOROUGH ==  'Brooklyn' ],
    name = 'Brooklyn',
    marker = dict(
        color = 'rgb(12, 12, 140)',
    )
)
trace3 = go.Box(
    y=df2.sale_price[df2.BOROUGH ==  'Queens' ],
    name = 'Queens',
    marker = dict(
        color = 'rgb(12, 128, 128)',
    )
)
trace4 = go.Box(
    y=df2.sale_price[df2.BOROUGH ==  'Staten Island' ],
    name = 'Staten Island',
    marker = dict(
        color = 'rgb(12, 12, 140)',
    )
)

dat = [trace0, trace1, trace2, trace3, trace4]
layout = go.Layout(
    title='Housing Prices by Boroughs',
    xaxis=dict(
        title='Borough'
    ),
    yaxis=dict(
        title='Sale Price'
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)'
)

fig = go.Figure(data=dat, layout=layout)
py.iplot(fig)

In [ ]:
df_bar=df[['sale_month', 'sale_price']].groupby(by='sale_month').count().sort_values(by='sale_month', ascending=True).reset_index()
df_bar.columns.values[1]='Sales_count'

plt.figure(figsize=(12,6))
sns.barplot(y = 'Sales_count', x = 'sale_month', data = df_bar)
plt.title('Sale Count by Months')
plt.show()

In [ ]:
# There is no seasonality for house prices. So we can delete theese columns.
del df["sale_year"]
del df["sale_month"]

In [ ]:
plt.figure(figsize=(16,6))
order = sorted(dataset['BUILDING CLASS CATEGORY'].unique())
sns.boxplot(x='BUILDING CLASS CATEGORY', y='sale_price', data=dataset, order=order)
plt.xticks(rotation=90)
plt.title('Sale Prices by Building Class Category')
plt.show()

In [ ]:
dataset = df[(df['COMMERCIAL UNITS']<20) & (df['TOTAL UNITS']<50) & (df['sale_price']<5000000) & (df['sale_price']>100000) & (df['GROSS SQUARE FEET']>0)]

plt.figure(figsize=(14,6))
sns.boxplot(x='COMMERCIAL UNITS', y="sale_price", data=dataset)
plt.title('Sale Prices by Commercial Units');

In [ ]:
# Deleting of unnecessary categorical variables for the model

del df["NEIGHBORHOOD"]
del df['BUILDING CLASS AT TIME OF SALE']
del df['TAX CLASS AT PRESENT']

In [ ]:
#Fixing all column names to be lowercase and no spaces
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.tax_class_at_time_of_sale = df.tax_class_at_time_of_sale.astype(object)

In [ ]:
X = df.drop(["sale_price","building_class_category","borough","tax_class_at_time_of_sale"],axis = 1).astype(int)
y = df["sale_price"].astype(int)

In [ ]:
X = df.drop(["sale_price","building_class_category","borough","tax_class_at_time_of_sale"],axis = 1).astype(int)
y = df["sale_price"].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [ ]:
################################# BASE MODEL
# Popular Regression Algorithms

lm = LinearRegression()
rf = RandomForestRegressor(random_state = 42)
lgbm = lightgbm.LGBMRegressor(random_state = 42)


algo = [lm, lgbm, rf]
result = []

for i in algo:
        start = time.process_time()
        model = i.fit(X_train,y_train)
        result.append([str(i).split("(")[0] + str("_baseline"), model.score(X_train, y_train), model.score(X_test, y_test),
                  np.sqrt(mean_squared_error(y_train, model.predict(X_train))),
                  np.sqrt(mean_squared_error(y_test, model.predict(X_test))),
                  mean_absolute_error(y_train, model.predict(X_train)),
                  mean_absolute_error(y_test, model.predict(X_test))])
        print(str(i).split("(")[0],"✓    ", "{}".format(time.process_time()-start),"sn")
        
result_raw_model = pd.DataFrame(result, columns = ["Algorithm", "Train_Score", "Test_Score", "Train_Rmse",
                                         "Test_Rmse", "Train_Mae", "Test_Mae"]).sort_values("Test_Rmse").set_index("Algorithm")
result_raw_model

In [ ]:
## Model Development
# 1 - a) Multicolinarity Control ( High correlation of independent variables with each other )¶


mtr = df.corr()
mask = np.zeros_like(mtr)
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(16,12))
plt.suptitle("Correlation Matrix of Columns in Heatmap", size=24)
sns.heatmap(mtr, mask= mask, annot=True, annot_kws={"size": 10});

In [ ]:
# Identifying and deleting variables with more than 75% correlation

corr_matrix = df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
[column for column in upper.columns if any(upper[column] > 0.75)]

In [ ]:
del df["residential_units"]
del df["total_units"]

In [ ]:
# Identifying and deleting variables that have nothing to do with the dependent variable
# 1 - b) Low Correlations with Dependent Variable

df.corr().abs().unstack().sort_values(ascending =False )["sale_price"]

In [ ]:
del df["building_age"]

In [ ]:
# Creating dummy variables from categorical variables

one_hot_variables = ["borough","building_class_category","tax_class_at_time_of_sale"]
df = pd.get_dummies(df,prefix = one_hot_variables, prefix_sep = "_" , drop_first = True)

In [ ]:
X = df.drop(["sale_price"],axis = 1).astype(int)
y = df["sale_price"].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [ ]:
algo = [lgbm,lm, rf]
result = []

for i in algo:
        start = time.process_time()
        model = i.fit(X_train,y_train)
        result.append([str(i).split("(")[0] + str("_processed"), model.score(X_train, y_train), model.score(X_test, y_test),
                  np.sqrt(mean_squared_error(y_train, model.predict(X_train))),
                  np.sqrt(mean_squared_error(y_test, model.predict(X_test))),
                  mean_absolute_error(y_train, model.predict(X_train)),
                  mean_absolute_error(y_test, model.predict(X_test))])
        print(str(i).split("(")[0],"✓    ", "{}".format(time.process_time()-start),"sec")
        
result_second_model = pd.DataFrame(result, columns = ["Algorithm", "Train_Score", "Test_Score", "Train_Rmse",
                                         "Test_Rmse", "Train_Mae", "Test_Mae"]).sort_values("Test_Rmse").set_index("Algorithm")

In [ ]:
result_second_model.head(2)

In [ ]:
result_raw_model.head(2)

In [ ]:
param_grid = {
    "n_estimators":[100,200,300],
    "max_depth":[10, 50, 100],
    "max_features":[6,8,10,12,14,16]
}

rf = RandomForestRegressor(random_state = 42)

rf_tuned = GridSearchCV(estimator = rf,
                            param_grid = param_grid,
                            cv = 2,
                            n_jobs=-1,
                        verbose=0)

rf_tuned.fit(X_train, y_train)
rf_tuned.best_estimator_

In [ ]:
model = rf_tuned.best_estimator_.fit(X_train,y_train) # Grid Search result

In [ ]:
result_optimize_model = []
result_optimize_model.append(["RandomForestRegressor Optimize", model.score(X_train, y_train), model.score(X_test, y_test),
                  np.sqrt(mean_squared_error(y_train, model.predict(X_train))),
                  np.sqrt(mean_squared_error(y_test, model.predict(X_test))),
                  mean_absolute_error(y_train, model.predict(X_train)),
                  mean_absolute_error(y_test, model.predict(X_test))])

pd.DataFrame(result_optimize_model, columns = ["Algorithm", "Train_Score", "Test_Score", "Train_Rmse",
                                         "Test_Rmse", "Train_Mae", "Test_Mae"]).sort_values("Test_Rmse").set_index("Algorithm")

In [ ]:
result_second_model[:1]


In [ ]:
rankings = model.feature_importances_.tolist()
importance = pd.DataFrame(sorted(zip(X_train.columns,rankings),reverse=True),columns=["variable","importance"]).sort_values("importance",ascending = False)

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(x="importance",
            y="variable",
            data=importance)
plt.title('Değişken Önemleri')
plt.tight_layout()